In [18]:
import pandas as pd 
import numpy as np 
import re
import pyarabic.araby as ar
import emoji
import string

In [92]:
df = pd.read_csv('10_Reviews.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     62884 non-null  int64 
 1   BusinessId     62884 non-null  int64 
 2   review_id      62884 non-null  int64 
 3   rating         62884 non-null  int64 
 4   comment        41021 non-null  object
 5   creation_date  62884 non-null  object
dtypes: int64(4), object(2)
memory usage: 2.9+ MB


In [93]:
df['creation_date'] = df['creation_date'].astype('datetime64[ns]')

In [6]:
df.isnull().sum()

Unnamed: 0           0
BusinessId           0
review_id            0
rating               0
comment          21863
creation_date        0
dtype: int64

In [94]:
df = df.dropna(subset=['comment'])

In [8]:
df.isnull().sum()

Unnamed: 0       0
BusinessId       0
review_id        0
rating           0
comment          0
creation_date    0
dtype: int64

In [95]:
df = df.drop('Unnamed: 0', axis=1)

In [66]:
df

,BusinessId,review_id,rating,comment,creation_date
0,60202,364189,1,كنت أتعامل معهم أسعارهم أرخص لكن آخر مرة حصلت لي عملية نصب على أساس الزباين غشيمين و جهلة اشتريت مكيف 2600 وحدة وسألت الموظف هل هو فعلا 2600 قال ايوا يابيه متأكد ولما جاء وقت التوصيل طلع 2400 وحدة أكلم موظف الفرع مايرد أكلم المشرف يقول طيب هي تفرق معك قمة الاستهتار علما ان الفاتورة مسجل 2600,2024-05-11 05:56:29.327
1,60202,356203,1,عدم المصداقية في اسعار المنتجات,2024-03-06 08:08:24.190
2,60202,317781,1,المتجر سئ جدا في مصداقيته في بيانات البضاعة وايضا في مواعيد التسليم غير صادقين ولاانصح بالتعامل معهم,2023-02-18 15:41:20.320
5,60202,285444,5,خدمة التوصيل سيئة للغاية ولا يوجد اي رد او تواصل مع العميل لا انصح بالتعامل معهم عن طريق المتجر الالكتروني تم رفع بلاغ تجاري,2022-09-03 14:01:18.933
6,60202,260277,5,تعاملهم ممتاز، وسرعة تنفيذ الطلب ممتازة، وحتى الشحن سريع ماشاء الله\r\nاهنيهم على شغلهم الرائع.,2022-05-21 13:07:39.523
...,...,...,...,...,...
62878,44461,61973,5,اتعب وانا اقول انها تجنن أخلاقها الحلوة وتعاملها الاحلى واهتمامها بأدق التفاصيل وغير التوصيل السريع الي تفقده بعض التاجرات❤️ وبعد الجوده الكويسة واكيددد مو اخر تعامل معها الله يوفقها يارب,2018-09-16 19:00:09.143
62879,44461,61446,5,اكثر مايميز المتجر سرعة الرد والتوصيل السريع وجودة المنتج واسعاره المنافسة .. باذن الله مابيكون اخر تعامل لي معاهم,2018-08-28 19:16:42.617
62880,44461,60746,5,التعامل مع متجر توليب تجربه تستحق اعادتها اكثر من مره من كل النواحي سرعة الرد و التعامل الراقي و التوصيل السريع و غير الاسعار الجميله و البضاعه الأجمل,2018-07-31 02:51:11.543
62881,44461,60443,5,تجربتي معها جمييله وانسانه ذوق واخلاق ومتعاونه جدا جدا ربي يسعدها ويوفقها ❤️❤️ من اهم المتاجر عندي مااستغني عنها 👍🏻,2018-07-22 16:43:45.897


### Comment split

In [96]:
# Define the delimiters for splitting, except period (.) at the end of the comment
delimiters = ['بس', 'لكن', 'ولكن', ',', '،']
pattern = '|'.join(map(re.escape, delimiters))

# Function to split the comment based on the delimiters, excluding period at the end
def split_comment(row):
    comment = row['comment'].strip()
    
    # Check if the comment ends with a period and remove it temporarily
    if comment.endswith('.'):
        comment = comment[:-1]
        ends_with_period = True
    else:
        ends_with_period = False
    
    # Split the comment using the defined delimiters
    split_comments = re.split(pattern, comment)
    split_comments = [c.strip() for c in split_comments if c.strip()]
    
    # Add the period back to the last segment if it was removed
    if ends_with_period and split_comments:
        split_comments[-1] += '.'

    # Return a new DataFrame with the duplicated row values and the split comments
    return pd.DataFrame({
        'BusinessId': [row['BusinessId']] * len(split_comments),
        'review_id': [row['review_id']] * len(split_comments),
        'rating': [row['rating']] * len(split_comments),
        'comment': split_comments,  # Use the split comments
        'creation_date': [row['creation_date']] * len(split_comments)
    })

# Apply the splitting function to each row and concatenate the results
df_split = pd.concat(df.apply(split_comment, axis=1).tolist())

df_split.reset_index(drop=True, inplace=True)

df_split

C:\Users\Dani_\AppData\Local\Temp\ipykernel_4172\3128071610.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_split = pd.concat(df.apply(split_comment, axis=1).tolist())


,BusinessId,review_id,rating,comment,creation_date
0,60202.0,364189.0,1.0,كنت أتعامل معهم أسعارهم أرخص,2024-05-11 05:56:29.327
1,60202.0,364189.0,1.0,آخر مرة حصلت لي عملية نصب على أساس الزباين غشيمين و جهلة اشتريت مكيف 2600 وحدة وسألت الموظف هل هو فعلا 2600 قال ايوا يابيه متأكد ولما جاء وقت التوصيل طلع 2400 وحدة أكلم موظف الفرع مايرد أكلم المشرف يقول طيب هي تفرق معك قمة الاستهتار علما ان الفاتورة مسجل 2600,2024-05-11 05:56:29.327
2,60202.0,356203.0,1.0,عدم المصداقية في اسعار المنتجات,2024-03-06 08:08:24.190
3,60202.0,317781.0,1.0,المتجر سئ جدا في مصداقيته في بيانات البضاعة وايضا في مواعيد التسليم غير صادقين ولاانصح بالتعامل معهم,2023-02-18 15:41:20.320
4,60202.0,285444.0,5.0,خدمة التوصيل سيئة للغاية ولا يوجد اي رد او تواصل مع العميل لا انصح بالتعامل معهم عن طريق المتجر الالكتروني تم رفع بلاغ تجاري,2022-09-03 14:01:18.933
...,...,...,...,...,...
51454,44461.0,61973.0,5.0,اتعب وانا اقول انها تجنن أخلاقها الحلوة وتعاملها الاحلى واهتمامها بأدق التفاصيل وغير التوصيل السريع الي تفقده بعض التاجرات❤️ وبعد الجوده الكويسة واكيددد مو اخر تعامل معها الله يوفقها يارب,2018-09-16 19:00:09.143
51455,44461.0,61446.0,5.0,اكثر مايميز المتجر سرعة الرد والتوصيل السريع وجودة المنتج واسعاره المنافسة .. باذن الله مابيكون اخر تعامل لي معاهم,2018-08-28 19:16:42.617
51456,44461.0,60746.0,5.0,التعامل مع متجر توليب تجربه تستحق اعادتها اكثر من مره من كل النواحي سرعة الرد و التعامل الراقي و التوصيل السريع و غير الاسعار الجميله و البضاعه الأجمل,2018-07-31 02:51:11.543
51457,44461.0,60443.0,5.0,تجربتي معها جمييله وانسانه ذوق واخلاق ومتعاونه جدا جدا ربي يسعدها ويوفقها ❤️❤️ من اهم المتاجر عندي مااستغني عنها 👍🏻,2018-07-22 16:43:45.897


In [14]:
pd.set_option('display.max_colwidth', None)

In [71]:
df_split.sample(20)

,BusinessId,review_id,rating,comment,creation_date
19128,51267.0,80150.0,5.0,ولي تعامل مرة اخرى 🌷,2019-06-02 22:42:37.003
32030,86755.0,198308.0,5.0,خدمة وتعامل ممتاز جداً جداً وسرعة بالتوصيل والله طلبت وصلتني اليوم الثاني على طول فعلاً شيء جميل ويستاهل الدعم الدائم وشكراً من القلب 💙,2021-03-27 14:11:34.220
23791,39976.0,209073.0,5.0,متميزين بعملهم ومصداقيه بعلاناتهم وخدمتهم رائعه,2021-05-16 23:14:53.610
15530,1045.0,126174.0,5.0,ممتاز,2020-05-28 19:28:55.177
38938,12375.0,17754.0,5.0,تجربتي معهم كانت متازة,2016-10-30 12:14:34.057
3122,95248.0,127679.0,5.0,متنة كثير لأني انضمت لها و اسعار الدورات مناسبه و جهود المدربين لا تنسى . شكرا,2020-06-06 20:20:21.110
39937,145217.0,182087.0,5.0,جوده المنتج ممتازه جدا.,2021-01-14 00:55:03.363
10949,148933.0,214859.0,5.0,الطلب ما أخذ أكثر من ٧ أيام,2021-06-21 16:35:48.287
8699,94622.0,133087.0,5.0,اشتريت من متجر ابعاد المحمول Case 280x شحن الى الرياض التغليف ممتاز جدا والتعامل راقي من الاخوة في المتجر ورد الاستفسارات بوقت قياسي.,2020-06-24 11:36:01.830
21080,20575.0,52361.0,5.0,مشاءالله سرعه بالتوصيل وصلتني في نفس اليوم الي حولت فيه غير الأسلوب والتعامل الطيب,2017-12-04 23:06:38.257


In [98]:
import re

def remove_repeated_characters(text):
    whitelist = ["الله", "ممتاز", "سامحهم", "معاهم", "ممتازه", "ممتازين", "مميزه", "لله", "ماشاءالله", "مشاءالله", "شاءالله", "والله"]
    
    # Check if text is None or not a string, return empty or original
    if not isinstance(text, str):
        return text
    
    # Split the text into words
    words = text.split()
    
    # Apply repeated character removal only to words not in the whitelist
    processed_words = [
        re.sub(r'(.)\1+', r'\1', word) if word not in whitelist else word
        for word in words
    ]
    
    return ' '.join(processed_words)

# Apply the function to the 'comment' column in the dataframe
df_split['comment'] = df_split['comment'].apply(remove_repeated_characters)


### Data cleaning

In [99]:
import numpy as np
import pandas as pd
import re
import pyarabic.araby as ar
import emoji
import string

# Clean newlines and strip text
def clean_comment(text):
    text = re.sub(r'[\r\n]', ' ', text)  # Replace newlines with spaces
    text = text.strip()  # Remove leading and trailing spaces
    return text

# Main cleaning function for removing URLs, numbers, etc.
def data_cleaning(text):
    if pd.isnull(text):  # Ensure that text is not NaN
        return text
    
    # Remove URLs
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    
    # Replace multiple whitespaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove numbers
    text = re.sub(r"\d+", " ", text)
    
    # Strip tashkeel and tatweel (diacritics and elongation)
    text = ar.strip_tashkeel(text)
    text = ar.strip_tatweel(text)
    
    # Replace special characters
    text = text.replace("#", "").replace("@", "").replace("_", "")  # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    
    # Remove emojis
    text = emoji.replace_emoji(text, replace="")  # Replace emojis with an empty string
    
    return text


# Normalize Arabic letters (e.g., "أ" -> "ا")
def normalize_arabic_letters(text):
    # Normalize Arabic letters
    text = text.replace("آ", "ا")
    text = text.replace("إ", "ا")
    text = text.replace("أ", "ا")
    text = text.replace("ؤ", "و")
    text = text.replace("ة", "ه")
    
    return text

# Remove any English letters
def remove_english(text):
    # Remove English characters (a-z, A-Z)
    return re.sub(r'[a-zA-Z]', '', text)

# Main data preprocessing pipeline
def data_preprocessing(data_frame):
    # Apply the cleaning function to the 'comment' column
    df_split['comment'] = df_split['comment'].apply(lambda x: data_cleaning(x))
    
    # Remove repeated letters
    #data_frame['comment'] = data_frame['comment'].apply(lambda x: remove_repeated_characters(x))
    
    # Normalize Arabic letters
    df_split['comment'] = df_split['comment'].apply(lambda x: normalize_arabic_letters(x))
    
    # Remove any English characters
    df_split['comment'] = df_split['comment'].apply(lambda x: remove_english(x))
    
    # Clean newlines and strip text
    df_split['comment'] = df_split['comment'].apply(clean_comment)
    
    return data_frame

# Example usage on your DataFrame
df_split = data_preprocessing(df_split)

In [100]:
df_split.sample(20)

,BusinessId,review_id,rating,comment,creation_date
18750,51267.0,162185.0,5.0,,2020-10-19 23:28:52.707
42043,15908.0,279411.0,5.0,حرفيا اخلاق عسل لو ما كنت تبغى تشتري بتصير تبغى,2022-08-04 21:12:23.207
44088,48246.0,96260.0,5.0,موقع جميل,2019-11-26 23:46:32.853
6205,226598.0,255703.0,5.0,الله يسعدهم,2022-04-12 21:26:23.187
10417,96402.0,103006.0,5.0,رائع وخدمه متازه اشياءهم ودايم في عروض وهذا احلى شي,2020-03-14 14:03:47.193
4759,289734.0,333167.0,5.0,الله يجزاهم خير,2023-05-10 08:48:13.053
42601,15908.0,189404.0,5.0,الطف انسانه ذوق بشكل تستحين تعطينها,2021-02-24 17:38:52.177
21869,39976.0,296024.0,5.0,اول مره اطلب من المتجر و ما شاء الله سرعه في التنفيذ و التوصيل الله يعطيكم العافيه على شغلكم الجبار,2022-10-18 08:31:25.717
51192,44461.0,78354.0,5.0,موقع جميل والتعامل محترم وراقي يعطيهم العافيه والله يوفقهم ويرزقهم ويسهل عليهم,2019-05-19 16:49:18.183
16443,238860.0,338866.0,5.0,شكرا شكرا بارلينا,2023-06-22 16:17:32.107


In [76]:
df_split.isnull().sum()

BusinessId       0
review_id        0
rating           0
comment          0
creation_date    0
dtype: int64

In [49]:
df.shape

(41021, 5)

In [101]:
df_split = df_split[~df_split['comment'].str.strip().eq('')]
df_split.reset_index(drop=True, inplace=True)
df_split

,BusinessId,review_id,rating,comment,creation_date
0,60202.0,364189.0,1.0,كنت اتعامل معهم اسعارهم ارخص,2024-05-11 05:56:29.327
1,60202.0,364189.0,1.0,اخر مره حصلت لي عمليه نصب على اساس الزباين غشيمين و جهله اشتريت مكيف وحده وسالت الموظف هل هو فعلا قال ايوا يابيه متاكد ولما جاء وقت التوصيل طلع وحده اكلم موظف الفرع مايرد اكلم المشرف يقول طيب هي تفرق معك قمه الاستهتار علما ان الفاتوره مسجل,2024-05-11 05:56:29.327
2,60202.0,356203.0,1.0,عدم المصداقيه في اسعار المنتجات,2024-03-06 08:08:24.190
3,60202.0,317781.0,1.0,المتجر سئ جدا في مصداقيته في بيانات البضاعه وايضا في مواعيد التسليم غير صادقين ولانصح بالتعامل معهم,2023-02-18 15:41:20.320
4,60202.0,285444.0,5.0,خدمه التوصيل سيئه لغايه ولا يوجد اي رد او تواصل مع العميل لا انصح بالتعامل معهم عن طريق المتجر الالكتروني تم رفع بلاغ تجاري,2022-09-03 14:01:18.933
...,...,...,...,...,...
50853,44461.0,61973.0,5.0,اتعب وانا اقول انها تجن اخلاقها الحلوه وتعاملها الاحلى واهتمامها بادق التفاصيل وغير التوصيل السريع الي تفقده بعض التاجرات وبعد الجوده الكويسه واكيد مو اخر تعامل معها الله يوفقها يارب,2018-09-16 19:00:09.143
50854,44461.0,61446.0,5.0,اكثر مايميز المتجر سرعه الرد والتوصيل السريع وجوده المنتج واسعاره المنافسه باذن الله مابيكون اخر تعامل لي معاهم,2018-08-28 19:16:42.617
50855,44461.0,60746.0,5.0,التعامل مع متجر توليب تجربه تستحق اعادتها اكثر من مره من كل النواحي سرعه الرد و التعامل الراقي و التوصيل السريع و غير الاسعار الجميله و البضاعه الاجمل,2018-07-31 02:51:11.543
50856,44461.0,60443.0,5.0,تجربتي معها جميله وانسانه ذوق واخلاق ومتعاونه جدا جدا ربي يسعدها ويوفقها من اهم المتاجر عندي ماستغني عنها,2018-07-22 16:43:45.897


In [102]:
df_split['text_length'] = df_split['comment'].map(lambda a: len(a))

C:\Users\Dani_\AppData\Local\Temp\ipykernel_4172\789469379.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_split['text_length'] = df_split['comment'].map(lambda a: len(a))


In [84]:
df_split['text_length'].max()

587

In [85]:
df_split['text_length'].min()

1

In [103]:
df_split = df_split[df_split['text_length']>4]

In [107]:
df_split.to_csv('Dania_cleaned_review_dataset.csv') 